In [61]:
import os
import sys
#import string
import shutil

### Generate the file paths to traverse, or a single path if a file name was given

In [62]:
def getfiles(path):
    if os.path.isdir(path):
        for root, dirs, files in os.walk(path):
            for name in files:
                yield os.path.join(root, name)
    else:
        yield path

### get source folder

In [63]:
fromdir = "/Users/ehaas/Documents/FHIR/test_template/pages"
#fromdir = input('enter the source folder: ')
print(fromdir)

/Users/ehaas/Documents/FHIR/test_template/pages


### get destination folder

In [64]:
destination = "/Users/ehaas/Documents/FHIR/test_template/test"
#destination = input('enter the destination folder: ')
print(destination)

/Users/ehaas/Documents/FHIR/test_template/test


### First Clear the directory

In [67]:
for filename in os.listdir(destination):
    file_path = os.path.join(destination,filename)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

### Flatten Directory and Replace 'index.md' with [Directory name].md

In [68]:
for f in getfiles(fromdir):
    
    filename = f.split('/')[-1]
    
    if filename != '.DS_Store':
        new_filepath = os.path.join(destination,filename)
        print(f'new_filepath before check: {new_filepath}')
        if os.path.isfile(new_filepath):
            filename = f.split('/')[-2] + '.md'
        new_filepath = os.path.join(destination,filename)
        print(f'new_filepath after check: {new_filepath}')
        shutil.copy(f, new_filepath)

new_filepath before check: /Users/ehaas/Documents/FHIR/test_template/test/index.md
new_filepath after check: /Users/ehaas/Documents/FHIR/test_template/test/index.md
new_filepath before check: /Users/ehaas/Documents/FHIR/test_template/test/index.md
new_filepath after check: /Users/ehaas/Documents/FHIR/test_template/test/Security.md
new_filepath before check: /Users/ehaas/Documents/FHIR/test_template/test/index.md
new_filepath after check: /Users/ehaas/Documents/FHIR/test_template/test/More Guidance.md
new_filepath before check: /Users/ehaas/Documents/FHIR/test_template/test/index.md
new_filepath after check: /Users/ehaas/Documents/FHIR/test_template/test/General Guidance.md
new_filepath before check: /Users/ehaas/Documents/FHIR/test_template/test/index.md
new_filepath after check: /Users/ehaas/Documents/FHIR/test_template/test/Operations and Search Parameters.md
new_filepath before check: /Users/ehaas/Documents/FHIR/test_template/test/index.md
new_filepath after check: /Users/ehaas/Docu